# Working with samplesets

First, we need a model to submit to the CQM Hybrid Solver through Leap.<br>
For this example, we will submit a CQM based on a Quadratic Assignment Problem.

### Build a CQM

In [ ]:
from dimod import ConstrainedQuadraticModel
import random

# Set size parameter. Model will have n^2 variables.
n=2

# Instantiate CQM and add grid of variables
cqm = ConstrainedQuadraticModel()
cqm.add_variables('BINARY', [(a,b) for a in range(n) for b in range(n)])

# Add psuedorandom dense objective.
random.seed(1)
cqm.set_objective([(v,w,random.randint(5,20)) for v in cqm.variables for w in cqm.variables])

# Add assignment constraint.
for i in range(n):
    cqm.add_constraint_from_iterable([((i,j), 1) for j in range(n)], '==', rhs=1, label=f'({i},*)')
    cqm.add_constraint_from_iterable([((j,i), 1) for j in range(n)], '==', rhs=1, label=f'(*, {i})')
    
print('Number of Variables:', len(cqm.variables))

### Get a sampleset by submitting the CQM to the solver

In [ ]:
from dwave.system import LeapHybridCQMSampler

cqm_sampler = LeapHybridCQMSampler()

sampleset = cqm_sampler.sample_cqm(cqm, time_limit=5)

#### Raw samplesets are best viewed with a print call

In [ ]:
print(sampleset)

#### Get some useful metrics for potential plots you might want to generate!<br>Reported times are measured in microseconds.

In [ ]:
for key, value in sampleset.info.items():
    print(key, value,'\n')

#### What about the solution quality?<br>We can look at the best solution in general (maybe infeasible) or filter for the best feasible solution found.

In [ ]:
# Best solution (objective minimized)
print(sampleset.first, "\n")

print("Sample energy: ", sampleset.first.energy, "\n")

print("Are all constraints satisfied? ", sampleset.first.is_feasible)

#### Let's filter for the best feasible sample - where all hard constraints are satisfied.

In [ ]:
feasible_sampleset = sampleset.filter(lambda d:d.is_feasible)

print('# Samples:', len(sampleset))
print('# Feasible:', len(feasible_sampleset), "\n")

print(feasible_sampleset)

In [ ]:
# The best feasible solution found is
best = feasible_sampleset.first

print("Sample energy: ", best.energy, "\n")

print("Are all constraints satisfied? ", best.is_feasible)

#### There are several ways to pass the information from a sampleset.

In [ ]:
# Looping through the data.
for datum in sampleset.data(fields=['sample', 'energy']):
    print(datum)

In [ ]:
# As a Dictionary.
dictionary = sampleset.data_vectors
print(dictionary.keys())

In [ ]:
# As a NumPy recarray.
array = sampleset.record
print(array[:3])

In [ ]:
# As a Pandas dataframe.
dataframe = sampleset.to_pandas_dataframe(True)

#### Pandas even has some of its own quick charting functions.

In [ ]:
dataframe.hist()

In [ ]:
dataframe.boxplot('energy')